##### **Aloitetaan tutkimalla ja puhdistamalla dataa(Esikäsittely - Preprocess)**
- Ladataan kaikki tarvittavat tiedosdot
- Määritetään nille polut jossa datasetit sijaitsee
- Teen funktion joka tulostaa **Shape, Columns, Info, datainfo, Summary, describe, null values, duplicates, missing values, duplicated rows**
- Kirjoitan koodin viereen välillä pieni info joka vois helpottaa ihmisiä lukemaan ja ymmärtämään paremmin

In [2]:
#Lataa tarvittavat kirjastot tähään soluun päivitä lopuksi tätä solua
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os # käytetään tiedostojen käsittelyyn ja hakemistojen hallintaan
from IPython.core.interactiveshell import InteractiveShell # tulostaa useamman tulosteen yhdellä kertaa 
InteractiveShell.ast_node_interactivity="all"
from datetime import datetime # aikaleimojen käsittelyyn 
import calendar # kalenterin käsittelyyn
import warnings # poistaa varoitusilmoitukset 
warnings.filterwarnings("ignore")

In [3]:
path = "/Users/emre/Desktop/ai_da_jamk/currentai/data/raw" # määritetään polku, jossa data sijaitsee 
contents = os.listdir(path)
print(f"Contents of '{path}': {contents}") # tulostetaan hakemiston tiedostot

Contents of '/Users/emre/Desktop/ai_da_jamk/currentai/data/raw': ['halfhourly_dataset', 'hhblock_dataset', 'daily_data.csv', 'darksky_parameters_documentation.html', 'acorn_details.csv', 'uk_bank_holidays.csv', 'daily_dataset', 'weather_daily_darksky.csv', 'informations_households.csv', 'daily_dataset.csv', 'halfhourly_data.csv', 'weather_hourly_darksky.csv', 'hhblock_data.csv']


In [4]:
BASE_DIR = '/Users/emre/Desktop/ai_da_jamk/currentai/data/raw'

files = {
    'household_info': os.path.join(BASE_DIR, 'informations_households.csv'),
    'acorn_details': os.path.join(BASE_DIR, 'acorn_details.csv'),
    'uk_bank_holidays': os.path.join(BASE_DIR, 'uk_bank_holidays.csv'),
    'weather_daily': os.path.join(BASE_DIR, 'weather_daily_darksky.csv'),
    'weather_hourly': os.path.join(BASE_DIR, 'weather_hourly_darksky.csv'),
    'daily_dataset': os.path.join(BASE_DIR, 'daily_dataset.csv')
}

DAILY_DIR = os.path.join(BASE_DIR, 'daily_dataset') # määritetään polku päivittäiselle datasetille
HALFHOURLY_DIR = os.path.join(BASE_DIR, 'halfhourly_dataset') # määritetään polku puolen tunnin datasetille
HHBLOCK_DIR = os.path.join(BASE_DIR, 'hhblock_dataset') # määritetään polku hhblock datasetille 

In [5]:
# funktio, joka lataa ja tulostaa datasetin tiedot 
def summarize_dataset(file_path, dataset_name):
    print(f"--- {dataset_name.upper()} ---")
    try:
        data = pd.read_csv(file_path, encoding='latin1')
        print(f"Shape: {data.shape}")
        print(f"Columns: {list(data.columns)}")
        print(f"Info:")
        print(data.info())
        print(f"Summary:")
        print(data.describe(include='all'))
        
        missing = data.isnull().sum()
        duplicates = data.duplicated().sum()
        print(f"Missing values per column:\n{missing}")
        print(f"Number of duplicate rows: {duplicates}")
        return data
    except Exception as e:
        print(f"Error reading {dataset_name}: {e}")

datasets = {}
for name, path in files.items():
    datasets[name] = summarize_dataset(path, name)

--- HOUSEHOLD_INFO ---
Shape: (5566, 5)
Columns: ['LCLid', 'stdorToU', 'Acorn', 'Acorn_grouped', 'file']
Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5566 entries, 0 to 5565
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   LCLid          5566 non-null   object
 1   stdorToU       5566 non-null   object
 2   Acorn          5566 non-null   object
 3   Acorn_grouped  5566 non-null   object
 4   file           5566 non-null   object
dtypes: object(5)
memory usage: 217.5+ KB
None
Summary:
            LCLid stdorToU    Acorn Acorn_grouped     file
count        5566     5566     5566          5566     5566
unique       5566        2       19             5      112
top     MAC005492      Std  ACORN-E      Affluent  block_0
freq            1     4443     1567          2192       50
Missing values per column:
LCLid            0
stdorToU         0
Acorn            0
Acorn_grouped    0
file             0
dtype: in

##### 1. **HOUSEHOLD_INFO**
  - 5566 kotitaluden tunnistetta **(LCLid)**
  - Jokainen talous kuuluu yhteen tai kahteen hinta luettoloon **(StdorToU) Std:Vakio, ToU: Aikaperusteinen**
  - Kotitaludet luokitellaan **(Acorn, Acorn_grouped)** kotitalauiden asemat yhteiskunnassa
  - **(file)** tarkoittaa mikä tieto sisältää kotitalouden tiedot. 112 uniikkia tiedostoa 
##### 2. **ACORN_DETAILS**
  - Acorn (A Classification Of Residential Neighbourhoods) on järjestelmä väestön luokitteluun
  - 826 riviä, joissa **pääkategoriat (MAIN CATEGORIES) ja alakategoriat (CATEGORIES)**
  - 17 numeerista saraketta **(esim. ACORN-A, ACORN-B)**
  - 1 puuttuva arvo sarakkeessa **REFERENCE**
##### 3. **UK_BANK_HOLIDAYS**
  - Lista Britannian pankki vapaista ja niiden tyypeistä
  - 25 pankki vapaata, kaikki uniikkeja
  - 11 kategoriaa, yleisin Boxing Day (3 esiintymää)
##### 4. **WEATHER_DAILY**
  - Päivittäiset säätiedot
  - 882 riviä sää ominaisuuksia, kuten lämpötila, pilvisyys, kosteus
  - Vähäinen määrä puuttuvia arvoja **(cloudCover, uvIndex)**
  - Tarkat tiedot lämpötiloista ja sääolosuhteista
##### 5. **WEATHER_HOURLY**
  - Tunnin välein kerättyjä säätietoja
  - 21,165 riviä ja 12 saraketta sääominaisuuksilla **(esim. lämpötila, näkyvyys)**
  - 13 puuttuvaa arvoa sarakkeessa **pressure**
  - **precipType: Kaksi luokkaa (sade, lumi)**
##### 6. **DAILY_DATASET**
  - Päivittäinen energiankulutus kotitalouksittain
  - 3,51 miljoonaa riviä ja 9 saraketta
  - Tietoja, kuten **(energy_mean, energy_sum, energy_max)**
  - 5566 uniikkia kotitaloutta, 829 päivää
  - Joitain puuttuvia arvoja energian kulutus tiedoissa
---
**Datan laatu:** Rakenteeltaan hyvä, vähän puuttuvia arvoja
**Yhdistämismahdollisuudet:**
  - LCLid yhdistää **(HOUSEHOLD_INFO, DAILY_DATASET)**
  - Säätiedot **(WEATHER_DAILY, WEATHER_HOURLY)** voidaan liittää päiväyksittäin
  - **(ACORN_DETAILS)** täydentävät kotitaloustietoja


**Etsitään alitiedosdot, lasketaan määrää ja yritetään saada yleiskatseus**
- Koodi etsii kaikki .csv tiedostot kolmesta eri kansiosta:
	•	Päivittäinen data (daily_dataset).
	•	Puolen tunnin data (halfhourly_dataset).
	•	HHBlock data (hhblock_dataset).
- Tulostaa jokaisesta kansiosta, montako .csv tiedostoa löytyi.
- Valitsen jokaisesta kansiosta yhden tiedoston (jos löytyy) ja analysoi sen summarize_dataset toiminnolla.

In [6]:
def get_all_csv_files(folder_path):
    """
    Kaivataan kaikki .csv tiedostot annetusta hakemistosta ja sen alihakemistoista.
    """
    csv_files = []
    for root, _, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.csv'):
                csv_files.append(os.path.join(root, file))
    return csv_files

In [7]:
daily_files = get_all_csv_files(DAILY_DIR) # haetaan kaikki päivittäiset datasetit
print(f"Number of daily dataset files: {len(daily_files)}")

halfhourly_files = get_all_csv_files(HALFHOURLY_DIR) # haetaan kaikki puolen tunnin datasetit
print(f"Number of half-hourly dataset files: {len(halfhourly_files)}")

hhblock_files = get_all_csv_files(HHBLOCK_DIR) # haetaan kaikki hhblock datasetit
print(f"Number of hhblock dataset files: {len(hhblock_files)}")

Number of daily dataset files: 112
Number of half-hourly dataset files: 112
Number of hhblock dataset files: 112


In [8]:
# analysoidaan yksi tiedosto päivittäisestä datasetistä 
if daily_files:
    sample_daily_data = summarize_dataset(daily_files[0], 'daily_dataset_sample')
else:
    print("No daily dataset files found.")

# analysoi yksi tiedosto puolen tunnin datasetistä
if halfhourly_files:
    sample_halfhourly_data = summarize_dataset(halfhourly_files[0], 'halfhourly_dataset_sample')
else:
    print("No half-hourly dataset files found.")

# analysoi yksi tiedosto hhblock datasetistä 
if hhblock_files:
    sample_hhblock_data = summarize_dataset(hhblock_files[0], 'hhblock_dataset_sample')
else:
    print("No hhblock dataset files found.")

--- DAILY_DATASET_SAMPLE ---
Shape: (29178, 9)
Columns: ['LCLid', 'day', 'energy_median', 'energy_mean', 'energy_max', 'energy_count', 'energy_std', 'energy_sum', 'energy_min']
Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29178 entries, 0 to 29177
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   LCLid          29178 non-null  object 
 1   day            29178 non-null  object 
 2   energy_median  29177 non-null  float64
 3   energy_mean    29177 non-null  float64
 4   energy_max     29177 non-null  float64
 5   energy_count   29178 non-null  int64  
 6   energy_std     29091 non-null  float64
 7   energy_sum     29177 non-null  float64
 8   energy_min     29177 non-null  float64
dtypes: float64(6), int64(1), object(2)
memory usage: 2.0+ MB
None
Summary:
            LCLid         day  energy_median   energy_mean    energy_max  \
count       29178       29178   29177.000000  29177.000000  29177.000000 

##### 1. **DAILY_DATASET_SAMPLE**
Näyte päivittäisestä energiankulutusdatasta
  - 29178 riviä ja 9 saraketta
  - Sisältää energiankulutuksen tilastot (esim. `energy_mean`, `energy_sum`, `energy_max`)
  - 50 kotitaloutta (`LCLid`) ja 815 eri päivää (`day`)
  - Joitain puuttuvia arvoja energiankulutustiedoissa (esim. `energy_std`: 87 puuttuvaa arvoa)
##### 2. **HALFHOURLY_DATASET_SAMPLE**
Näyte puolen tunnin välein kerätystä energiankulutuksesta
  - 1,395,110 riviä ja 3 saraketta
  - Sarakkeet: `LCLid`, aikaleima (`tstp`), energiankulutus (`energy(kWh/hh)`)
  - 50 kotitaloutta ja 39,098 uniikkia aikaleimaa
  - Ei puuttuvia arvoja
##### 3. **HHBLOCK_DATASET_SAMPLE**
Näyte kotitalouksien puolen tunnin energiankulutuksesta päiväkohtaisesti
  - 28,836 riviä ja 50 saraketta
  - Sarakkeet: `LCLid`, päivä (`day`), ja 48 aikaväliä (`hh_0` - `hh_47`)
  - 50 kotitaloutta ja 813 eri päivää
  - Sarakkeessa `hh_30` 48 puuttuvaa arvoa
---
1. **Datan laatu:** 
   - Kokonaisuudessaan hyvin rakenteellista ja vain vähän puuttuvia arvoja.
   - **DAILY_DATASET_SAMPLE** ja **HHBLOCK_DATASET_SAMPLE** sisältävät pieniä määriä puuttuvia arvoja, jotka on käsiteltävä ennen analyysia.
2. **Yhdistämismahdollisuudet:**
   - `LCLid` toimii avainsarakkeena yhdistettäessä datasetit.
   - Päivämäärä (`day`) ja aikaleima (`tstp`) mahdollistavat ajallisen analyysin ja yhdistämisen muihin tietoihin.

## **1.HOUSEHOLD INFO**

In [10]:
household_info_path = files['household_info']
household_info = pd.read_csv(household_info_path)

print(household_info.info())
print(household_info.describe())
print(household_info.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5566 entries, 0 to 5565
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   LCLid          5566 non-null   object
 1   stdorToU       5566 non-null   object
 2   Acorn          5566 non-null   object
 3   Acorn_grouped  5566 non-null   object
 4   file           5566 non-null   object
dtypes: object(5)
memory usage: 217.5+ KB
None
            LCLid stdorToU    Acorn Acorn_grouped     file
count        5566     5566     5566          5566     5566
unique       5566        2       19             5      112
top     MAC005492      Std  ACORN-E      Affluent  block_0
freq            1     4443     1567          2192       50
       LCLid stdorToU    Acorn Acorn_grouped     file
0  MAC005492      ToU   ACORN-        ACORN-  block_0
1  MAC001074      ToU   ACORN-        ACORN-  block_0
2  MAC000002      Std  ACORN-A      Affluent  block_0
3  MAC003613      Std  ACORN-A     

In [11]:
print(household_info.isnull().sum()) # tarkistetaan puuttuvat arvot

LCLid            0
stdorToU         0
Acorn            0
Acorn_grouped    0
file             0
dtype: int64


In [12]:
household_info['stdorToU'] = household_info['stdorToU'].str.strip().str.upper() # muutetaan kaikki kirjaimet isoiksi kirjaimiksi stdorToU sarakkeessa 
household_info['Acorn'] = household_info['Acorn'].str.strip() # poistetaan välilyönnit Acorn sarakkeesta 
household_info['Acorn_grouped'] = household_info['Acorn_grouped'].str.strip() # poistetaan välilyönnit Acorn_grouped sarakkeesta

In [13]:
household_info = household_info.drop_duplicates() # poistetaan duplikaatit 
household_info = household_info[household_info['Acorn_grouped'].notnull()] # poistetaan puuttuvat arvot Acorn_grouped sarakkeesta

In [14]:
household_info['Acorn_grouped'] = household_info['Acorn_grouped'].replace({ # korvataan Acorn_grouped sarakkeen arvot
    'Affluent': 'High Income', # korvataan Affluent High Income arvolla
    'Comfortable': 'Middle Income', # korvataan Comfortable Middle Income arvolla
    'Adversity': 'Low Income' # korvataan Adversity Low Income arvolla
})

In [15]:
household_info = household_info.reset_index(drop=True) # nollataan indeksi

In [16]:
household_info.to_csv('data/processed/cleaned_household_info.csv', index=False) # tallennetaan puhdistettu datasetti

OSError: Cannot save file into a non-existent directory: 'data/processed'

## **2.ACORN DETAILS**

In [ ]:
acorn_details_path = "data/raw/acorn_details.csv"
acorn_details = pd.read_csv(acorn_details_path, encoding='latin1')

print(acorn_details.info())
print(acorn_details.describe())
print(acorn_details.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 826 entries, 0 to 825
Data columns (total 20 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   MAIN CATEGORIES  826 non-null    object 
 1   CATEGORIES       826 non-null    object 
 2   REFERENCE        825 non-null    object 
 3   ACORN-A          826 non-null    float64
 4   ACORN-B          826 non-null    float64
 5   ACORN-C          826 non-null    float64
 6   ACORN-D          826 non-null    float64
 7   ACORN-E          826 non-null    float64
 8   ACORN-F          826 non-null    float64
 9   ACORN-G          826 non-null    float64
 10  ACORN-H          826 non-null    float64
 11  ACORN-I          826 non-null    float64
 12  ACORN-J          826 non-null    float64
 13  ACORN-K          826 non-null    float64
 14  ACORN-L          826 non-null    float64
 15  ACORN-M          826 non-null    float64
 16  ACORN-N          826 non-null    float64
 17  ACORN-O         

In [ ]:
acorn_details.isnull().sum() # tarkistetaan puuttuvat arvot
acorn_details.duplicated().sum() # tarkistetaan duplikaatit

MainCategories    0
SubCategories     0
Reference         0
AcornA            0
AcornB            0
AcornC            0
AcornD            0
AcornE            0
AcornF            0
AcornG            0
AcornH            0
AcornI            0
AcornJ            0
AcornK            0
AcornL            0
AcornM            0
AcornN            0
AcornO            0
AcornP            0
AcornQ            0
dtype: int64

np.int64(0)

In [ ]:
acorn_details = acorn_details.drop_duplicates() # poistetaan duplikaatit

In [ ]:
acorn_details['REFERENCE'] = acorn_details['REFERENCE'].fillna('Unknown') # täytetään puuttuvat arvot Unknown arvolla

In [ ]:
acorn_details['MAIN CATEGORIES'] = acorn_details['MAIN CATEGORIES'].str.strip().str.title() # muutetaan kaikki kirjaimet isoiksi kirjaimiksi ja poistetaan ylimääräiset välilyönnit MAIN CATEGORIES sarakkeessa
acorn_details['CATEGORIES'] = acorn_details['CATEGORIES'].str.strip().str.title() # muutetaan kaikki kirjaimet isoiksi kirjaimiksi ja poistetaan ylimääräiset välilyönnit CATEGORIES sarakkeessa
acorn_details['REFERENCE'] = acorn_details['REFERENCE'].str.strip().str.title() # muutetaan kaikki kirjaimet isoiksi kirjaimiksi ja poistetaan ylimääräiset välilyönnit REFERENCE sarakkeessa

In [ ]:
acorn_details = acorn_details.rename(columns={ # muutetaan sarakkeiden nimet
    'MAIN CATEGORIES': 'MainCategories', # muutetaan MAIN CATEGORIES sarakkeen nimi MainCategories
    'CATEGORIES': 'SubCategories', # muutetaan CATEGORIES sarakkeen nimi SubCategories
    'REFERENCE': 'Reference', # muutetaan REFERENCE sarakkeen nimi Reference
    'ACORN-A': 'AcornA',
    'ACORN-B': 'AcornB',
    'ACORN-C': 'AcornC',
    'ACORN-D': 'AcornD',
    'ACORN-E': 'AcornE',
    'ACORN-F': 'AcornF',
    'ACORN-G': 'AcornG',
    'ACORN-H': 'AcornH',
    'ACORN-I': 'AcornI',
    'ACORN-J': 'AcornJ',
    'ACORN-K': 'AcornK',
    'ACORN-L': 'AcornL',
    'ACORN-M': 'AcornM',
    'ACORN-N': 'AcornN',
    'ACORN-O': 'AcornO',
    'ACORN-P': 'AcornP',
    'ACORN-Q': 'AcornQ'
})

In [ ]:
numeric_columns = [col for col in acorn_details.columns if col.startswith('Acorn')] # valitaan sarakkeet, jotka alkavat Acorn kirjaimella 
for col in numeric_columns:
    acorn_details[col] = pd.to_numeric(acorn_details[col], errors='coerce') # muutetaan sarakkeiden arvot numeerisiksi 

In [ ]:
for col in numeric_columns: 
    q1 = acorn_details[col].quantile(0.25) 
    q3 = acorn_details[col].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    acorn_details = acorn_details[(acorn_details[col] >= lower_bound) & (acorn_details[col] <= upper_bound)] # poistetaan outlierit

In [ ]:
acorn_details = acorn_details.reset_index(drop=True) # nollataan indeksi

In [ ]:
acorn_details.to_csv('data/processed/cleaned_acorn_details.csv', index=False) # tallennetaan puhdistettu datasetti

## **3.UK BANK HOLIDAYS**

In [ ]:
uk_bank_holidays_path = "data/raw/uk_bank_holidays.csv"
uk_bank_holidays = pd.read_csv(uk_bank_holidays_path)

print(uk_bank_holidays.info())
print(uk_bank_holidays.describe())
print(uk_bank_holidays.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Bank holidays  25 non-null     object
 1   Type           25 non-null     object
dtypes: object(2)
memory usage: 528.0+ bytes
None
       Bank holidays        Type
count             25          25
unique            25          11
top       2012-12-26  Boxing Day
freq               1           3
  Bank holidays                                          Type
0    2012-12-26                                    Boxing Day
1    2012-12-25                                 Christmas Day
2    2012-08-27                           Summer bank holiday
3    2012-05-06  Queen?s Diamond Jubilee (extra bank holiday)
4    2012-04-06          Spring bank holiday (substitute day)


In [ ]:
uk_bank_holidays.isnull().sum() # tarkistetaan puuttuvat arvot
uk_bank_holidays.duplicated().sum() # tarkistetaan duplikaatit

Bank holidays    0
dtype: int64

np.int64(0)

In [ ]:
uk_bank_holidays['Bank holidays']= uk_bank_holidays['Bank holidays'].str.strip().str.title() # muutetaan kaikki kirjaimet isoiksi kirjaimiksi ja poistetaan ylimääräiset välilyönnit Bank holidays sarakkeessa
uk_bank_holidays['Type']= uk_bank_holidays['Type'].str.strip().str.title() # muutetaan kaikki kirjaimet isoiksi kirjaimiksi ja poistetaan ylimääräiset välilyönnit Type sarakkeessa

In [ ]:
uk_bank_holidays['Type'] = uk_bank_holidays['Type'].str.replace('?', "'", regex=False) # korvataan ? merkki ' merkillä Type sarakkeessa 

In [ ]:
uk_bank_holidays = uk_bank_holidays.drop_duplicates() # poistetaan duplikaatit 

In [ ]:
uk_bank_holidays = pd.to_datetime(uk_bank_holidays['Bank holidays'], errors='coerce') # muutetaan Bank holidays sarakkeen arvot päivämäärämuotoon

In [ ]:
if isinstance(uk_bank_holidays, pd.Series):
    uk_bank_holidays = uk_bank_holidays.to_frame() # muutetaan Series DataFrameksi 

In [ ]:
uk_bank_holidays['Bank holidays'] = pd.to_datetime(uk_bank_holidays['Bank holidays'], errors='coerce') # muutetaan Bank holidays sarakkeen arvot päivämäärämuotoon

In [ ]:
uk_bank_holidays = uk_bank_holidays.sort_values(by='Bank holidays').reset_index(drop=True) # järjestetään arvot Bank holidays sarakkeen mukaan ja nollataan indeksi


In [ ]:
uk_bank_holidays.to_csv('data/processed/cleaned_uk_bank_holidays.csv', index=False) # tallennetaan puhdistettu datasetti 

## **4.WEATHER DAILY**

In [ ]:
weather_daily_path = "data/raw/weather_daily_darksky.csv"
weather_daily = pd.read_csv(weather_daily_path)

print(weather_daily.info())
print(weather_daily.describe())
print(weather_daily.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 882 entries, 0 to 881
Data columns (total 32 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   temperatureMax               882 non-null    float64
 1   temperatureMaxTime           882 non-null    object 
 2   windBearing                  882 non-null    int64  
 3   icon                         882 non-null    object 
 4   dewPoint                     882 non-null    float64
 5   temperatureMinTime           882 non-null    object 
 6   cloudCover                   881 non-null    float64
 7   windSpeed                    882 non-null    float64
 8   pressure                     882 non-null    float64
 9   apparentTemperatureMinTime   882 non-null    object 
 10  apparentTemperatureHigh      882 non-null    float64
 11  precipType                   882 non-null    object 
 12  visibility                   882 non-null    float64
 13  humidity            

In [ ]:
weather_daily.isnull().sum() # tarkistetaan puuttuvat arvot
weather_daily.duplicated().sum() # tarkistetaan duplikaatit

temperatureMax                 0
temperatureMaxTime             0
windBearing                    0
icon                           0
dewPoint                       0
temperatureMinTime             0
cloudCover                     1
windSpeed                      0
pressure                       0
apparentTemperatureMinTime     0
apparentTemperatureHigh        0
precipType                     0
visibility                     0
humidity                       0
apparentTemperatureHighTime    0
apparentTemperatureLow         0
apparentTemperatureMax         0
uvIndex                        1
time                           0
sunsetTime                     0
temperatureLow                 0
temperatureMin                 0
temperatureHigh                0
sunriseTime                    0
temperatureHighTime            0
uvIndexTime                    1
summary                        0
temperatureLowTime             0
apparentTemperatureMin         0
apparentTemperatureMaxTime     0
apparentTe

np.int64(0)

In [ ]:
weather_daily['cloudCover'] = weather_daily['cloudCover'].fillna(weather_daily['cloudCover'].median()) # täytetään puuttuvat arvot cloudCover sarakkeessa mediaanilla
weather_daily['uvIndex'] = weather_daily['uvIndex'].fillna(weather_daily['uvIndex'].median()) # täytetään puuttuvat arvot uvIndex sarakkeessa mediaanilla 

In [ ]:
weather_daily['time'] = pd.to_datetime(weather_daily['time']) # muutetaan time sarakkeen arvot päivämäärämuotoon
weather_daily['sunriseTime'] = pd.to_datetime(weather_daily['sunriseTime']) # muutetaan sunriseTime sarakkeen arvot päivämäärämuotoon
weather_daily['sunsetTime'] = pd.to_datetime(weather_daily['sunsetTime']) # muutetaan sunsetTime sarakkeen arvot päivämäärämuotoon
weather_daily['temperatureMaxTime'] = pd.to_datetime(weather_daily['temperatureMaxTime']) # muutetaan temperatureMaxTime sarakkeen arvot päivämäärämuotoon
weather_daily['temperatureMinTime'] = pd.to_datetime(weather_daily['temperatureMinTime']) # muutetaan temperatureMinTime sarakkeen arvot päivämäärämuotoon

In [ ]:
weather_daily = weather_daily.drop_duplicates() # poistetaan duplikaatit

In [ ]:
numeric_columns = weather_daily.select_dtypes(include=['float64', 'int64']).columns # valitaan numeeriset sarakkeet 
weather_daily[numeric_columns] = weather_daily[numeric_columns].apply(pd.to_numeric, errors='coerce') # muutetaan numeeriset sarakkeet numeerisiksi

In [ ]:
weather_daily = weather_daily.rename(columns={ # muutetaan sarakkeiden nimet
    'time': 'Date',
    'temperatureMax': 'TempMax',
    'temperatureMin': 'TempMin',
    'temperatureHigh': 'TempHigh',
    'temperatureLow': 'TempLow',
    'humidity': 'Humidity',
    'windSpeed': 'WindSpeed',
    'precipType': 'PrecipitationType',
    'cloudCover': 'CloudCover',
    'moonPhase': 'MoonPhase'
})
print(weather_daily.columns)

Index(['TempMax', 'temperatureMaxTime', 'windBearing', 'icon', 'dewPoint',
       'temperatureMinTime', 'CloudCover', 'WindSpeed', 'pressure',
       'apparentTemperatureMinTime', 'apparentTemperatureHigh',
       'PrecipitationType', 'visibility', 'Humidity',
       'apparentTemperatureHighTime', 'apparentTemperatureLow',
       'apparentTemperatureMax', 'uvIndex', 'Date', 'sunsetTime', 'TempLow',
       'TempMin', 'TempHigh', 'sunriseTime', 'temperatureHighTime',
       'uvIndexTime', 'summary', 'temperatureLowTime',
       'apparentTemperatureMin', 'apparentTemperatureMaxTime',
       'apparentTemperatureLowTime', 'MoonPhase', 'DaylightDuration'],
      dtype='object')


In [ ]:
for col in weather_daily.select_dtypes(include=['float64', 'int64']).columns:
    q1 = weather_daily[col].quantile(0.25)
    q3 = weather_daily[col].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    weather_daily = weather_daily[(weather_daily[col] >= lower_bound) & (weather_daily[col] <= upper_bound)] # poistetaan outlierit
    print(f"Column: {col}, Lower bound: {lower_bound}, Upper bound: {upper_bound}") 

Column: TempMax, Lower bound: -3.014999999999999, Upper bound: 31.185
Column: windBearing, Lower bound: 29.5, Upper bound: 393.5
Column: dewPoint, Lower bound: -7.015000000000002, Upper bound: 20.745000000000005
Column: CloudCover, Lower bound: -0.01624999999999982, Upper bound: 0.9337499999999997
Column: WindSpeed, Lower bound: -0.5462500000000001, Upper bound: 7.723750000000001
Column: pressure, Lower bound: 987.1562499999998, Upper bound: 1041.8662500000003
Column: apparentTemperatureHigh, Lower bound: -8.661249999999995, Upper bound: 34.76875
Column: visibility, Lower bound: 8.419999999999998, Upper bound: 15.620000000000001
Column: Humidity, Lower bound: 0.5149999999999999, Upper bound: 1.035
Column: apparentTemperatureLow, Lower bound: -13.465, Upper bound: 27.295
Column: apparentTemperatureMax, Lower bound: -6.522499999999999, Upper bound: 33.777499999999996
Column: uvIndex, Lower bound: -3.5, Upper bound: 8.5
Column: TempLow, Lower bound: -6.965, Upper bound: 23.395
Column: Tem

In [ ]:
weather_daily['DaylightDuration'] = (weather_daily['sunsetTime'] - weather_daily['sunriseTime']).dt.total_seconds() / 3600 # lasketaan päivänvalon kesto tuntien mukaan 
print(weather_daily['DaylightDuration'].head()) # tulostetaan päivänvalon kesto 

1   7.931
2   7.847
3   8.169
4   7.828
5   7.867
Name: DaylightDuration, dtype: float64


In [ ]:
weather_daily.to_csv('data/processed/cleaned_weather_daily.csv', index=False) # tallennetaan puhdistettu datasetti

## **5.WEATHER HOURLY**

In [ ]:
weather_hourly_path = "data/raw/weather_hourly_darksky.csv"
weather_hourly = pd.read_csv(weather_hourly_path)

print(weather_hourly.info())
print(weather_hourly.describe())
print(weather_hourly.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21165 entries, 0 to 21164
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   visibility           21165 non-null  float64
 1   windBearing          21165 non-null  int64  
 2   temperature          21165 non-null  float64
 3   time                 21165 non-null  object 
 4   dewPoint             21165 non-null  float64
 5   pressure             21152 non-null  float64
 6   apparentTemperature  21165 non-null  float64
 7   windSpeed            21165 non-null  float64
 8   precipType           21165 non-null  object 
 9   icon                 21165 non-null  object 
 10  humidity             21165 non-null  float64
 11  summary              21165 non-null  object 
dtypes: float64(7), int64(1), object(4)
memory usage: 1.9+ MB
None
       visibility  windBearing  temperature  dewPoint  pressure  \
count   21165.000    21165.000    21165.000 21165.000 211

In [ ]:
print(weather_hourly.isnull().sum()) # tarkistetaan puuttuvat arvot

visibility              0
windBearing             0
temperature             0
time                    0
dewPoint                0
pressure               13
apparentTemperature     0
windSpeed               0
precipType              0
icon                    0
humidity                0
summary                 0
dtype: int64


In [ ]:
weather_hourly['pressure'] = weather_hourly['pressure'].fillna(weather_hourly['pressure'].median()) # täytetään puuttuvat arvot pressure sarakkeessa mediaanilla koska se on jatkuva muuttuja 

In [ ]:
weather_hourly['time'] = pd.to_datetime(weather_hourly['time']) # muutetaan time sarakkeen arvot päivämäärämuotoon

In [ ]:
weather_hourly['precipType'] = weather_hourly['precipType'].str.strip().str.title() # muutetaan kaikki kirjaimet isoiksi kirjaimiksi ja poistetaan ylimääräiset välilyönnit precipType sarakkeessa
weather_hourly['icon'] = weather_hourly['icon'].str.strip().str.title() # muutetaan kaikki kirjaimet isoiksi kirjaimiksi ja poistetaan ylimääräiset välilyönnit icon sarakkeessa
weather_hourly['summary'] = weather_hourly['summary'].str.strip().str.title() # muutetaan kaikki kirjaimet isoiksi kirjaimiksi ja poistetaan ylimääräiset välilyönnit summary sarakkeessa

In [ ]:
def detect_outliers(column):
    q1 = weather_hourly[column].quantile(0.25)
    q3 = weather_hourly[column].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    outliers = weather_hourly[(weather_hourly[column] < lower_bound) | (weather_hourly[column] > upper_bound)] # etsitään outlierit
    print(f"Outliers detected in {column}: {len(outliers)}") 
    return lower_bound, upper_bound 

lower_temp, upper_temp = detect_outliers('temperature')


Outliers detected in temperature: 162


In [ ]:
weather_hourly = weather_hourly[(weather_hourly['temperature'] >= lower_temp) &  # poistetaan outlierit
                                (weather_hourly['temperature'] <= upper_temp)]  

print("Shape of dataset after removing outliers in temperature:", weather_hourly.shape)

Shape of dataset after removing outliers in temperature: (21003, 12)


In [ ]:
weather_hourly['windChillEffect'] = weather_hourly['apparentTemperature'] - weather_hourly['temperature'] # lasketaan tuulen viilentävä vaikutus 
print(weather_hourly[['temperature', 'apparentTemperature', 'windChillEffect']].head()) # tulostetaan lämpötila, tuntuu kuin lämpötila ja tuulen viilentävä vaikutus 

   temperature  apparentTemperature  windChillEffect
0       10.240               10.240            0.000
1        9.760                8.240           -1.520
2        9.460                7.760           -1.700
3        9.230                7.440           -1.790
4        9.260                7.240           -2.020


In [ ]:
weather_hourly.to_csv('data/processed/cleaned_weather_hourly.csv', index=False) # tallennetaan puhdistettu datasetti

## **6.DAILY DATASET**

In [ ]:
daily_dataset_path = "data/raw/daily_dataset.csv"
daily_dataset = pd.read_csv(daily_dataset_path)

print(daily_dataset.info())
print(daily_dataset.describe())
print(daily_dataset.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3510433 entries, 0 to 3510432
Data columns (total 9 columns):
 #   Column         Dtype  
---  ------         -----  
 0   LCLid          object 
 1   day            object 
 2   energy_median  float64
 3   energy_mean    float64
 4   energy_max     float64
 5   energy_count   int64  
 6   energy_std     float64
 7   energy_sum     float64
 8   energy_min     float64
dtypes: float64(6), int64(1), object(2)
memory usage: 241.0+ MB
None
       energy_median  energy_mean  energy_max  energy_count  energy_std  \
count    3510403.000  3510403.000 3510403.000   3510433.000 3499102.000   
mean           0.159        0.212       0.835        47.804       0.173   
std            0.170        0.191       0.668         2.811       0.153   
min            0.000        0.000       0.000         0.000       0.000   
25%            0.067        0.098       0.346        48.000       0.069   
50%            0.115        0.163       0.688        48.000  

In [ ]:
print(daily_dataset.isnull().sum()) # tarkistetaan puuttuvat arvot

LCLid                0
day                  0
energy_median       30
energy_mean         30
energy_max          30
energy_count         0
energy_std       11331
energy_sum          30
energy_min          30
dtype: int64


In [ ]:
columns_with_missing = ['energy_median', 'energy_mean', 'energy_max', 'energy_sum', 'energy_min', 'energy_std'] # valitaan sarakkeet, joissa on puuttuvia arvoja 
for col in columns_with_missing: # täytetään puuttuvat arvot mediaanilla koska sarakkeet ovat vinoja
    daily_dataset[col] = daily_dataset[col].fillna(daily_dataset[col].median())

print(daily_dataset.isnull().sum())

LCLid            0
day              0
energy_median    0
energy_mean      0
energy_max       0
energy_count     0
energy_std       0
energy_sum       0
energy_min       0
dtype: int64


In [ ]:
daily_dataset['day'] = pd.to_datetime(daily_dataset['day'])

In [ ]:
daily_dataset = daily_dataset.drop_duplicates()
print(daily_dataset.shape)

(3510433, 9)


In [ ]:
def detect_outliers(column):
    q1 = daily_dataset[column].quantile(0.25)
    q3 = daily_dataset[column].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    outliers = daily_dataset[(daily_dataset[column] < lower_bound) | (daily_dataset[column] > upper_bound)] # etsitään outlierit
    print(f"Outliers detected in {column}: {len(outliers)}") 
    return lower_bound, upper_bound

lower_sum, upper_sum = detect_outliers('energy_sum') # etsitään outlierit energy_sum sarakkeesta

Outliers detected in energy_sum: 201571


In [ ]:
daily_dataset = daily_dataset[(daily_dataset['energy_sum'] >= lower_sum) & (daily_dataset['energy_sum'] <= upper_sum)]
print(daily_dataset.shape)

(3308862, 9)


In [ ]:
daily_dataset['energy_range'] = daily_dataset['energy_max'] - daily_dataset['energy_min'] # lasketaan energian vaihteluväli 
print(daily_dataset[['energy_max', 'energy_min', 'energy_range']].head()) # tulostetaan energian maksimi, minimi ja vaihteluväli

   energy_max  energy_min  energy_range
0       0.868       0.072         0.796
1       1.116       0.031         1.085
2       0.685       0.064         0.621
3       0.676       0.065         0.611
4       0.788       0.066         0.722


In [ ]:
daily_data.to_csv('data/processed/cleaned_daily_dataset.csv', index=False) # tallennetaan puhdistettu datasetti 

In [ ]:
daily_data = pd.concat([pd.read_csv(file) for file in daily_files], ignore_index=True) # yhdistetään kaikki päivittäiset datasetit yhdeksi datasetiksi koska ne ovat samassa muodossa
print(f"Shape of concatenated daily dataset: {daily_data.shape}") 

Shape of concatenated daily dataset: (3510433, 9)


In [ ]:
halfhourly_data = pd.concat([pd.read_csv(file) for file in halfhourly_files], ignore_index=True) # yhdistetään kaikki puolen tunnin datasetit yhdeksi datasetiksi koska ne ovat samassa muodossa
print(f"Shape of concatenated half-hourly dataset: {halfhourly_data.shape}")

Shape of concatenated half-hourly dataset: (167817021, 3)


In [ ]:
hhblock_data = pd.concat([pd.read_csv(file) for file in hhblock_files], ignore_index=True) # yhdistetään kaikki hhblock datasetit yhdeksi datasetiksi koska ne ovat samassa muodossa
print(f"Shape of concatenated hhblock dataset: {hhblock_data.shape}")

Shape of concatenated hhblock dataset: (3469352, 50)


In [ ]:
daily_data.to_csv("data/raw/daily_data.csv", index=False) # tallennetaan yhdistetty datasetti
halfhourly_data.to_csv("data/raw/halfhourly_data.csv", index=False) # tallennetaan yhdistetty datasetti
hhblock_data.to_csv("data/raw/hhblock_data.csv", index=False) # tallennetaan yhdistetty datasetti

## **6.2 DAILY DATA**

In [ ]:
daily_data_path = "data/raw/daily_data.csv"
daily_data = pd.read_csv(daily_data_path)

print(daily_data.info())
print(daily_data.describe())
print(daily_data.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3510433 entries, 0 to 3510432
Data columns (total 9 columns):
 #   Column         Dtype  
---  ------         -----  
 0   LCLid          object 
 1   day            object 
 2   energy_median  float64
 3   energy_mean    float64
 4   energy_max     float64
 5   energy_count   int64  
 6   energy_std     float64
 7   energy_sum     float64
 8   energy_min     float64
dtypes: float64(6), int64(1), object(2)
memory usage: 241.0+ MB
None
       energy_median  energy_mean  energy_max  energy_count  energy_std  \
count    3510403.000  3510403.000 3510403.000   3510433.000 3499102.000   
mean           0.159        0.212       0.835        47.804       0.173   
std            0.170        0.191       0.668         2.811       0.153   
min            0.000        0.000       0.000         0.000       0.000   
25%            0.067        0.098       0.346        48.000       0.069   
50%            0.115        0.163       0.688        48.000  

In [ ]:
print(daily_data.isnull().sum()) # tarkistetaan puuttuvat arvot

LCLid                0
day                  0
energy_median       30
energy_mean         30
energy_max          30
energy_count         0
energy_std       11331
energy_sum          30
energy_min          30
dtype: int64


In [ ]:
columns_with_missing = ['energy_median', 'energy_mean', 'energy_max', 'energy_sum', 'energy_min', 'energy_std']
for col in columns_with_missing: # täytetään puuttuvat arvot mediaanilla koska sarakkeet ovat vinoja 
    daily_data[col] = daily_data[col].fillna(daily_data[col].median())
print(daily_data.isnull().sum())

LCLid            0
day              0
energy_median    0
energy_mean      0
energy_max       0
energy_count     0
energy_std       0
energy_sum       0
energy_min       0
dtype: int64


In [ ]:
daily_data['day'] = pd.to_datetime(daily_data['day']) # muutetaan day sarakkeen arvot päivämäärämuotoon

In [ ]:
daily_data.duplicated().sum() # tarkistetaan duplikaatit
daily_data.shape # tulostetaan datasetin muoto 

np.int64(0)

(3510433, 9)

In [ ]:
def detect_outliers(column): # funktio, joka etsii outlierit
    q1 = daily_data[column].quantile(0.25)
    q3 = daily_data[column].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    outliers = daily_data[(daily_data[column] < lower_bound) | (daily_data[column] > upper_bound)]
    print(f"Outliers detected in {column}: {len(outliers)}")
    return lower_bound, upper_bound
outliers = detect_outliers('energy_sum') # etsitään outlierit energy_sum sarakkeesta
print(outliers)

Outliers detected in energy_sum: 201571
(np.float64(-7.1485), np.float64(24.399500000000003))


In [ ]:
daily_data[(daily_data['energy_sum'] >= lower_sum) & (daily_data['energy_sum'] <= upper_sum)] # poistetaan outlierit
print(daily_data.shape)

,LCLid,day,energy_median,energy_mean,energy_max,energy_count,energy_std,energy_sum,energy_min
0,MAC000027,2011-12-07,0.147,0.160,0.408,25,0.062,3.997,0.103
1,MAC000027,2011-12-08,0.170,0.193,0.459,48,0.077,9.287,0.105
2,MAC000027,2011-12-09,0.144,0.157,0.365,48,0.052,7.525,0.101
3,MAC000027,2011-12-10,0.241,0.264,0.814,48,0.135,12.684,0.101
4,MAC000027,2011-12-11,0.278,0.288,0.540,48,0.113,13.845,0.126
...,...,...,...,...,...,...,...,...,...
3510428,MAC004883,2013-05-11,0.048,0.065,0.155,48,0.033,3.123,0.007
3510429,MAC004883,2013-05-12,0.053,0.089,0.805,48,0.125,4.279,0.041
3510430,MAC004883,2013-05-13,0.046,0.053,0.120,48,0.023,2.530,0.006
3510431,MAC004883,2013-05-14,0.049,0.062,0.112,48,0.025,2.968,0.041


(3510433, 9)


In [ ]:
daily_data['energy_range'] = daily_data['energy_max'] - daily_data['energy_min'] # lasketaan energian vaihteluväli
print(daily_data[['energy_max', 'energy_min', 'energy_range']].head())

   energy_max  energy_min  energy_range
0       0.408       0.103         0.305
1       0.459       0.105         0.354
2       0.365       0.101         0.264
3       0.814       0.101         0.713
4       0.540       0.126         0.414


In [ ]:
daily_data.to_csv('data/processed/cleaned_daily_data.csv', index=False) # tallennetaan puhdistettu datasetti

## **7. HALFHOURLY DATA**

In [ ]:
halfhourly_data_path = "data/raw/halfhourly_data.csv"
halfhourly_data = pd.read_csv(halfhourly_data_path)

print(halfhourly_data.info())
print(halfhourly_data.describe())
print(halfhourly_data.head())

NameError: name 'pd' is not defined

In [ ]:
print(halfhourly_data.isnull().sum()) # tarkistetaan puuttuvat arvot
print(halfhourly_data.duplicated().sum()) # tarkistetaan duplikaatit

LCLid             0
tstp              0
energy(kWh/hh)    0
dtype: int64
0


In [ ]:
print(halfhourly_data['energy(kWh/hh)'].unique()[:10]) # tulostetaan ainutlaatuiset arvot energy(kWh/hh) sarakkeesta

[' 0.185 ' ' 0.155 ' ' 0.147 ' ' 0.164 ' ' 0.187 ' ' 0.118 ' ' 0.114 '
 ' 0.135 ' ' 0.138 ' ' 0.107 ']


In [ ]:
halfhourly_data['energy(kWh/hh)'] = halfhourly_data['energy(kWh/hh)'].str.strip() # poistetaan ylimääräiset välilyönnit energy(kWh/hh) sarakkeesta
halfhourly_data['energy(kWh/hh)'] = pd.to_numeric(halfhourly_data['energy(kWh/hh)'], errors='coerce') # muutetaan energy(kWh/hh) sarakkeen arvot numeerisiksi

In [ ]:
halfhourly_data['energy(kWh/hh)'] = halfhourly_data['energy(kWh/hh)'].fillna(halfhourly_data['energy(kWh/hh)'].median())
# täytetään puuttuvat arvot mediaanilla koska sarakkeet ovat vinoja 

In [ ]:
halfhourly_data['tstp'] = pd.to_datetime(halfhourly_data['tstp'], errors='coerce') 
# muutetaan tstp sarakkeen arvot päivämäärämuotoon

In [ ]:
halfhourly_data = halfhourly_data.drop_duplicates() # poistetaan duplikaatit

In [ ]:
def detect_outliers(column): # funktio, joka etsii outlierit
    q1 = halfhourly_data[column].quantile(0.25)
    q3 = halfhourly_data[column].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    outliers = halfhourly_data[(halfhourly_data[column] < lower_bound) | (halfhourly_data[column] > upper_bound)]
    print(f"Outliers detected in {column}: {len(outliers)}")
    return lower_bound, upper_bound
outliers = detect_outliers('energy(kWh/hh)') # etsitään outlierit energy(kWh/hh) sarakkeesta
print(outliers) 

Outliers detected in energy(kWh/hh): 15416021
(np.float64(-0.21349999999999997), np.float64(0.5105))


In [ ]:
halfhourly_data = halfhourly_data[(halfhourly_data['energy(kWh/hh)'] >= lower_sum) & (halfhourly_data['energy(kWh/hh)'] <= upper_sum)] # poistetaan outlierit
print(halfhourly_data.shape) 

(167817021, 3)


In [ ]:
halfhourly_data.to_csv('data/processed/cleaned_halfhourly_data.csv', index=False) # tallennetaan puhdistettu datasetti

## **8.1 HHBLOCK DATA**

In [ ]:
hhblock_data_path = "data/raw/hhblock_data.csv"
hhblock_data = pd.read_csv(hhblock_data_path)

print(hhblock_data.info())
print(hhblock_data.describe())
print(hhblock_data.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3469352 entries, 0 to 3469351
Data columns (total 50 columns):
 #   Column  Dtype  
---  ------  -----  
 0   LCLid   object 
 1   day     object 
 2   hh_0    float64
 3   hh_1    float64
 4   hh_2    float64
 5   hh_3    float64
 6   hh_4    float64
 7   hh_5    float64
 8   hh_6    float64
 9   hh_7    float64
 10  hh_8    float64
 11  hh_9    float64
 12  hh_10   float64
 13  hh_11   float64
 14  hh_12   float64
 15  hh_13   float64
 16  hh_14   float64
 17  hh_15   float64
 18  hh_16   float64
 19  hh_17   float64
 20  hh_18   float64
 21  hh_19   float64
 22  hh_20   float64
 23  hh_21   float64
 24  hh_22   float64
 25  hh_23   float64
 26  hh_24   float64
 27  hh_25   float64
 28  hh_26   float64
 29  hh_27   float64
 30  hh_28   float64
 31  hh_29   float64
 32  hh_30   float64
 33  hh_31   float64
 34  hh_32   float64
 35  hh_33   float64
 36  hh_34   float64
 37  hh_35   float64
 38  hh_36   float64
 39  hh_37   float64
 40  

In [ ]:
print(hhblock_data.isnull().sum()) # tarkistetaan puuttuvat arvot

LCLid       0
day         0
hh_0        0
hh_1        0
hh_2        0
hh_3        0
hh_4        0
hh_5        0
hh_6        0
hh_7        0
hh_8        0
hh_9        0
hh_10       0
hh_11       0
hh_12       0
hh_13       0
hh_14       0
hh_15       0
hh_16       0
hh_17       0
hh_18       0
hh_19       2
hh_20       0
hh_21       0
hh_22       0
hh_23       0
hh_24       0
hh_25      21
hh_26       2
hh_27       0
hh_28       0
hh_29       0
hh_30    5460
hh_31       0
hh_32       0
hh_33       0
hh_34       0
hh_35       0
hh_36       1
hh_37       0
hh_38       0
hh_39       0
hh_40       0
hh_41       0
hh_42       0
hh_43       0
hh_44       0
hh_45       0
hh_46       0
hh_47       0
dtype: int64


In [ ]:
hh_columns = [col for col in hhblock_data.columns if col.startswith('hh_')] # valitaan sarakkeet, jotka alkavat hh_ kirjaimella
hhblock_data[hh_columns] = hhblock_data[hh_columns].fillna(hhblock_data[hh_columns].median()) # täytetään puuttuvat arvot mediaanilla koska sarakkeet ovat vinoja
print(hhblock_data.isnull().sum()) # tarkistetaan puuttuvat arvot

LCLid    0
day      0
hh_0     0
hh_1     0
hh_2     0
hh_3     0
hh_4     0
hh_5     0
hh_6     0
hh_7     0
hh_8     0
hh_9     0
hh_10    0
hh_11    0
hh_12    0
hh_13    0
hh_14    0
hh_15    0
hh_16    0
hh_17    0
hh_18    0
hh_19    0
hh_20    0
hh_21    0
hh_22    0
hh_23    0
hh_24    0
hh_25    0
hh_26    0
hh_27    0
hh_28    0
hh_29    0
hh_30    0
hh_31    0
hh_32    0
hh_33    0
hh_34    0
hh_35    0
hh_36    0
hh_37    0
hh_38    0
hh_39    0
hh_40    0
hh_41    0
hh_42    0
hh_43    0
hh_44    0
hh_45    0
hh_46    0
hh_47    0
dtype: int64


In [ ]:
hhblock_data['day'] = pd.to_datetime(hhblock_data['day'], errors='coerce') # muutetaan day sarakkeen arvot päivämäärämuotoon

In [ ]:
hhblock_data = hhblock_data.drop_duplicates() # poistetaan duplikaatit
hhblock_data.shape

(3469352, 50)

In [ ]:
def detect_outliers(column): # funktio, joka etsii outlierit
    q1 = hhblock_data[column].quantile(0.25)
    q3 = hhblock_data[column].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    return lower_bound, upper_bound

outlier_columns = {} # tallennetaan outlierit sanakirjaan
for col in hh_columns:
    lower, upper = detect_outliers(col) # etsitään outlierit
    outlier_columns[col] = (lower, upper)
    print(f"{col}: Outliers below {lower:.2f}, above {upper:.2f}")

hh_0: Outliers below -0.16, above 0.40
hh_1: Outliers below -0.13, above 0.35
hh_2: Outliers below -0.11, above 0.31
hh_3: Outliers below -0.10, above 0.28
hh_4: Outliers below -0.09, above 0.26
hh_5: Outliers below -0.08, above 0.25
hh_6: Outliers below -0.08, above 0.24
hh_7: Outliers below -0.08, above 0.24
hh_8: Outliers below -0.08, above 0.24
hh_9: Outliers below -0.08, above 0.24
hh_10: Outliers below -0.09, above 0.26
hh_11: Outliers below -0.10, above 0.28
hh_12: Outliers below -0.12, above 0.32
hh_13: Outliers below -0.15, above 0.38
hh_14: Outliers below -0.17, above 0.43
hh_15: Outliers below -0.19, above 0.48
hh_16: Outliers below -0.21, above 0.51
hh_17: Outliers below -0.21, above 0.52
hh_18: Outliers below -0.21, above 0.52
hh_19: Outliers below -0.21, above 0.52
hh_20: Outliers below -0.21, above 0.51
hh_21: Outliers below -0.21, above 0.51
hh_22: Outliers below -0.21, above 0.51
hh_23: Outliers below -0.21, above 0.51
hh_24: Outliers below -0.22, above 0.51
hh_25: Out

In [ ]:
for col, (lower, upper) in outlier_columns.items(): # poistetaan outlierit
    hhblock_data = hhblock_data[(hhblock_data[col] >= lower) & (hhblock_data[col] <= upper)]
hhblock_data.shape


(1371160, 50)

In [ ]:
hhblock_data['daily_sum'] = hhblock_data[hh_columns].sum(axis=1) # lasketaan päivittäinen summamäärä
hhblock_data['daily_mean'] = hhblock_data[hh_columns].mean(axis=1) # lasketaan päivittäinen keskiarvo
hhblock_data['daily_max'] = hhblock_data[hh_columns].max(axis=1) # lasketaan päivittäinen maksimi
hhblock_data['daily_min'] = hhblock_data[hh_columns].min(axis=1) # lasketaan päivittäinen minimi
hhblock_data['daily_std'] = hhblock_data[hh_columns].std(axis=1) # lasketaan päivittäinen keskihajonta

In [ ]:
hhblock_data.to_csv('data/processed/cleaned_hhblock_data.csv', index=False) # tallennetaan puhdistettu datasetti

## Tulo- ja ACORN-ryhmien Käyttö Tärkeimmissä Kategorioissa

| **Kategoria**         | **Eniten Käyttävä Tulo** | **Vähiten Käyttävä Tulo** | **Eniten Käyttävä ACORN** | **Vähiten Käyttävä ACORN** |
|------------------------|--------------------------|---------------------------|---------------------------|----------------------------|
| **Rahoitus**          | Korkea tulotaso          | Matala tulotaso           | ACORN-B                   | ACORN-U                    |
| **Digitaalisuus**      | Korkea tulotaso          | Matala tulotaso           | ACORN-C                   | ACORN-U                    |
| **Perhe**             | Keskitulotaso            | Matala tulotaso           | ACORN-A                   | ACORN-U                    |
| **Koulutus**          | Korkea tulotaso          | Matala tulotaso           | ACORN-A                   | ACORN-Q                    |
| **Terveys**           | Matala tulotaso          | Korkea tulotaso           | ACORN-E                   | ACORN-L                    |
| **Liikenne**          | Matala tulotaso          | Korkea tulotaso           | ACORN-O                   | ACORN-K                    |
| **Ympäristö**         | Keskitulotaso            | Matala tulotaso           | ACORN-H                   | ACORN-Q                    |
| **Yhteisön turvallisuus** | Keskitulotaso        | Korkea tulotaso           | ACORN-J                   | ACORN-U                    |
| **Ostoskanavat**      | Korkea tulotaso          | Matala tulotaso           | ACORN-F                   | ACORN-U                    |
| **Vapaa-aika**        | Korkea tulotaso          | Matala tulotaso           | ACORN-C                   | ACORN-U                    |

## Yhteenveto
- **Korkeat tulot**: Hyödyntävät eniten **Rahoitusta**, **Digitaalisuutta** ja **Koulutusta**.
- **Keskitulot**: Tasapainoinen käyttö; erityisesti **Perhe** ja **Ympäristö** korostuvat.
- **Matala tulotaso**: Käyttö keskittyy **Terveys**, **Liikenne** ja **Yhteisön turvallisuus** -kategorioihin.